In [31]:
import numpy as np
import cv2
import dtcwt
import random
import os

# Create Folder

In [32]:
if not os.path.exists("image"):
    os.makedirs("image")

# Global Parameter

In [33]:
step = 5
alpha = 5
lowpass_str = 5
highpass_str = 5
embed_channel = 1 # 1: u; 2: v
code_length = 60
bit_to_pixel = 2 # each bit will generate bit_to_pixel*bit_to_pixel pixels
wm_level = 4
key = 8745
random_placement_key = 9260
coverimgpath = "frame_0000.png"


In [34]:
# calculate the wm size that this cover image can afford 
image = cv2.imread(coverimgpath)
# this is equal to celling(h/8)
wm_w = (((((image.shape[1] + 1) // 2 + 1) // 2 + 1) // 2 + 1) // 2 + 1) // 2 
wm_h = (((((image.shape[0] + 1) // 2 + 1) // 2 + 1) // 2 + 1) // 2 + 1) // 2
if wm_w % 2 == 1:
    wm_w += 1
if wm_h % 2 == 1:
    wm_h += 1

print(f'Key Image Size (w * h): {wm_w}  {wm_h}')

Key Image Size (w * h): 60  34


# Generate Bitstream and Corresponding Key Image

In [35]:
def generate_random_binary_string(length, random_seed=None):
    if random_seed is not None:
        random.seed(random_seed)
    
    binary_string = ''.join(random.choice('10') for _ in range(length))
    return binary_string

def generate_image(random_binary_string, width, height, n):
    # Create a black image
    image = np.zeros((height, width), dtype=np.uint8)

    # Initialize variables for tracking the position in the binary string
    index = 0
    row = 0
    col = 0

    # Loop until the image is completely filled
    while row + n <= height:
        # Check if we have scanned all bits; if so, reset the index
        if index >= len(random_binary_string):
            index = 0

        # Determine the color based on the current bit (0 or 1)
        color = 0 if random_binary_string[index] == "0" else 255

        # Move to the next position
        if col + n <= width:
            # Place the nxn block
            for i in range(n):
                for j in range(n):
                    x = col + j
                    y = row + i
                    image[y, x] = color
            col += n
            index += 1
        else:
            # print(index)
            col = 0
            row += n

    return image

def get_random_pos(big_matrix_shape, small_matrices, seed=None):
    # Set seed for reproducibility
    random.seed(seed)
    # Keep track of occupied positions
    occupied = np.zeros(big_matrix_shape, dtype=bool)

    random_position_list = []
    
    for small_matrix in small_matrices:
        small_height, small_width = small_matrix.shape
        
        # Get all possible positions for this small matrix
        possible_positions = [(i, j) for i in range(big_matrix_shape[0] - small_height + 1) 
                                      for j in range(big_matrix_shape[1] - small_width + 1)
                                      if not np.any(occupied[i:i+small_height, j:j+small_width])]
        

        # If no possible position, return an error
        if not possible_positions:
            raise ValueError("No space left for the matrix of shape {}".format(small_matrix.shape))
        
        # Randomly select one position
        chosen_position = random.choice(possible_positions)
        
        # Place the small matrix at the chosen position
        # big_matrix[chosen_position[0]:chosen_position[0]+small_height, 
        #            chosen_position[1]:chosen_position[1]+small_width] = small_matrix

        random_position_list.append((chosen_position[0], chosen_position[0]+small_height, chosen_position[1], chosen_position[1]+small_width))
        
        # Mark the position as occupied
        occupied[chosen_position[0]:chosen_position[0]+small_height, 
                 chosen_position[1]:chosen_position[1]+small_width] = True
        
    return random_position_list

def place_random_pos(big_matrix, small_matrices, random_pos):
    
    for idx, small_matrix in enumerate(small_matrices):
       
        # Place the small matrix at the chosen position
        big_matrix[random_pos[idx][0]:random_pos[idx][1], 
                   random_pos[idx][2]:random_pos[idx][3]] = small_matrix

        
    return big_matrix

def create_symmetrical_array(large_width, large_height, small_array):
    # Validate input dimensions
    if large_width < small_array.shape[1] * 2 or large_height < small_array.shape[0] * 2:
        raise ValueError("Large array must be at least twice as large as the small array in both dimensions.")

    # Duplicate and mirror the small array
    mirrored_small = np.block([[small_array, small_array[:, ::-1]],
                               [small_array[::-1, :], small_array[::-1, ::-1]]])

    # Initialize the large array with ones
    large_array = np.ones((large_height, large_width), dtype=small_array.dtype)

    # Calculate the start and end indices for placing the small arrays
    start_x, end_x = 0, small_array.shape[1]
    start_y, end_y = 0, small_array.shape[0]

    # Place the small arrays at the corners of the large array
    large_array[start_y:end_y, start_x:end_x] = small_array
    large_array[start_y:end_y, -end_x:] = small_array[:, ::-1]
    large_array[-end_y:, start_x:end_x] = small_array[::-1, :]
    large_array[-end_y:, -end_x:] = small_array[::-1, ::-1]

    return large_array

def extract_and_average_small_arrays(large_array, small_width, small_height):
    # Extract the four small arrays
    small_array_1 = large_array[:small_height, :small_width]
    small_array_2 = large_array[:small_height, -small_width:]
    small_array_3 = large_array[-small_height:, :small_width]
    small_array_4 = large_array[-small_height:, -small_width:]

    # Normalize orientation of the small arrays
    small_array_2 = small_array_2[:, ::-1]
    small_array_3 = small_array_3[::-1, :]
    small_array_4 = small_array_4[::-1, ::-1]

    # Calculate the average of the four small arrays
    average_array = (small_array_1 + small_array_2 + small_array_3 + small_array_4) / 4

    return average_array



In [36]:
random_binary_string = generate_random_binary_string(code_length, key)
image = generate_image(random_binary_string, int(wm_w/2), int(wm_h/2), bit_to_pixel)
key_image = create_symmetrical_array(wm_w, wm_h, image)
cv2.imwrite("image/wm.png", key_image) # This is the key image


True

# Polynomial for Threshold Generation while Reading Bitstream from Key Image

In [37]:
x = np.array([201.8083448840181, 201.65635643248135,201.37170672855643,200.8592226964692,199.23487024944833,195.25194811126022,191.8126761620028,187.44035977431795,181.6775642641012,174.43329279410978,165.85723363933954,161.47919075367426,156.72325717258732,152.02736535769264,147.76242540533113,143.30003808913443,139.07401350683818,135.21552660969684,131.17617535127852,126.6786417491157,123.00818411501238,118.72886262596417,115.70126254295224,111.97272383002851,107.99399966101403,101.66663601565126,94.18656941872487,71.03652862307095,])
y = np.array([93,87,87,90,90,87,87,84,81,66,69,75,66,75,72,69,57,63,60,54,54,45,54,54,42,42,36,30,])

# Fit a polynomial of degree 1 (linear)
coefficients = np.polyfit(x, y, 2)

# Create a polynomial function from the coefficients
polynomial = np.poly1d(coefficients)

# Utility Functions

In [38]:
def rebin(a, shape):
    if a.shape[0] % 2 == 1:
        a = np.vstack((a, np.zeros((1, a.shape[1]))))
    sh = shape[0], a.shape[0] // shape[0], shape[1], a.shape[1] // shape[1]
    # print(sh)
    return a.reshape(sh).mean(-1).mean(1)

def recover_string_from_image(n, original_length, gray_image):
    # Initialize variables for position tracking
    row, col = 0, 0

    # Use a list to store the detected strings in multiple passes
    detected_strings = []

    while row + n <= gray_image.shape[0]:  # Ensure we don't go beyond image boundaries
        # Initialize an empty string for the current pass
        current_string = ""

        while len(current_string) < original_length and row + n <= gray_image.shape[0]:
            # Extract the n x n block
            block = gray_image[row : row + n, col : col + n]

            avg_color = np.mean(block)

            # current_string += '0' if avg_color < 127.5 else '1'

            black_appear = 0
            look_all_black = True
            for i in range(n):
                for j in range(n):
                    if block[i, j] < 20:
                        black_appear += 1
                    if block[i, j] > 50:
                        look_all_black = False

            if black_appear > 0 and avg_color < 127.5:
                current_string += "0"
            else:
                current_string += "1"

            # Move to the next position
            col += n
            if col + n > gray_image.shape[1]:  # If we are at the end of a row
                col = 0
                row += n

        while len(current_string) < original_length:
            current_string += "5"

        detected_strings.append(current_string)

    # Calculate the final binary string using voting for each position
    final_string = ""
    for i in range(original_length):
        ones = sum([1 for s in detected_strings if s[i] == "1"])
        zeros = sum([1 for s in detected_strings if s[i] == "0"])

        # If there are more ones than zeros, append '1', else append '0'.
        # If there's a tie, append a random choice
        if ones > zeros:
            final_string += "1"
        elif zeros > ones:
            final_string += "0"
        else:
            final_string += np.random.choice(["0", "1"])

    return final_string

def embed_frame(frame, wm_coeffs):
    img = cv2.cvtColor(frame.astype(np.float32), cv2.COLOR_BGR2YUV)

    # # DTCWT for V channel
    v_transform = dtcwt.Transform2d()
    v_coeffs = v_transform.forward(img[:, :, 2], nlevels=3)

    # DTCWT for U channel
    img_transform = dtcwt.Transform2d()
    img_coeffs = img_transform.forward(img[:, :, 1], nlevels=3)

    # DTCWT for Y Channel
    y_transform = dtcwt.Transform2d()
    y_coeffs = y_transform.forward(img[:, :, 0], nlevels=3)

    # # Masks for the level 3 subbands
    masks3 = [0 for i in range(6)]
    shape3 = y_coeffs.highpasses[2][:, :, 0].shape

    # embed watermark highpass into U channel's highpass
    for i in range(6):
        masks3[i] = cv2.filter2D(
            np.abs(y_coeffs.highpasses[1][:, :, i]),
            -1,
            np.array([[1 / 4, 1 / 4], [1 / 4, 1 / 4]]),
        )
        masks3[i] = np.ceil(rebin(masks3[i], shape3) * (1 / step))
        # print(masks3[i])
        masks3[i] *= 1.0 / max(12.0, np.amax(masks3[i]))
        # print(masks3[i].shape) (135, 240)

    for i in range(6):
        coeffs = np.zeros(masks3[i].shape, dtype="complex_")

        coeffs_height, coeffs_width = coeffs.shape
        height_moving_offset = wm_coeffs.lowpass.shape[0] + 1
        width_moving_offset = wm_coeffs.lowpass.shape[1] + 1
        

        if coeffs_height % 2 == 1:
            height_up = int(coeffs_height/2) - height_moving_offset
            height_down = int(coeffs_height/2) + 1 + height_moving_offset
        else:
            height_up = int(coeffs_height/2) - height_moving_offset
            height_down = int(coeffs_height/2) + height_moving_offset

        if coeffs_width % 2 == 1:
            width_left = int(coeffs_width/2) - width_moving_offset
            width_right = int(coeffs_width/2) + 1 + width_moving_offset
        else:
            width_left = int(coeffs_width/2) - width_moving_offset
            width_right = int(coeffs_width/2) + width_moving_offset

        for lv in range(wm_level):
            w = 0
            h = 0
            for m in range(lv):
                w += wm_coeffs.highpasses[m][:, :, i].shape[1]
                h += wm_coeffs.highpasses[m][:, :, i].shape[0]


            coeffs[
                height_up - h - wm_coeffs.highpasses[lv][:, :, i].shape[0] : height_up - h,
                width_left - w - wm_coeffs.highpasses[lv][:, :, i].shape[1] : width_left - w,
            ] = wm_coeffs.highpasses[lv][:, :, i]

            coeffs[
                height_up - h - wm_coeffs.highpasses[lv][:, :, i].shape[0] : height_up - h,
                width_right + w: width_right + w + wm_coeffs.highpasses[lv][:, :, i].shape[1],
            ] = wm_coeffs.highpasses[lv][:, :, i]

            coeffs[
                height_down + h: height_down + h + wm_coeffs.highpasses[lv][:, :, i].shape[0] ,
                width_left - w - wm_coeffs.highpasses[lv][:, :, i].shape[1] : width_left - w,
            ] = wm_coeffs.highpasses[lv][:, :, i]

            coeffs[
                height_down + h: height_down + h + wm_coeffs.highpasses[lv][:, :, i].shape[0] ,
                width_right + w: width_right + w + wm_coeffs.highpasses[lv][:, :, i].shape[1],
            ] = wm_coeffs.highpasses[lv][:, :, i]


        img_coeffs.highpasses[2][:, :, i] += highpass_str * (masks3[i] * coeffs)

    ### embed watermark lowpass into V channel's highpass[2] (highpass as mask)
    lowpass_masks = [0 for i in range(6)]

    for i in range(4):
        lowpass_masks[i] = cv2.filter2D(
            np.abs(
                np.abs(y_coeffs.highpasses[1][:, :, i]),
            ),
            -1,
            np.array([[1 / 4, 1 / 4], [1 / 4, 1 / 4]]),
        )
        lowpass_masks[i] = np.ceil(
            rebin(lowpass_masks[i], v_coeffs.highpasses[2].shape) * (1 / step)
        )
        lowpass_masks[i] *= 1.0 / max(12.0, np.amax(lowpass_masks[i]))

    # lv1_h = wm_coeffs.highpasses[0][:, :, i].shape[0]
    # lv1_w = wm_coeffs.highpasses[0][:, :, i].shape[1]
    
    coeffs_height, coeffs_width = y_coeffs.highpasses[2][:, :, 0].shape

    if coeffs_height % 2 == 1:
        height_up = int(coeffs_height/2) - 1
        height_down = int(coeffs_height/2) 
    else:
        height_up = int(coeffs_height/2) - 1
        height_down = int(coeffs_height/2) - 1

    if coeffs_width % 2 == 1:
        width_left = int(coeffs_width/2) - 1
        width_right = int(coeffs_width/2)  
    else:
        width_left = int(coeffs_width/2) - 1
        width_right = int(coeffs_width/2) - 1

    coeff = wm_coeffs.lowpass

    if coeff.shape[0] % 2 != y_coeffs.highpasses[2][:, :, 0].shape[0] % 2:
        temp = np.zeros((coeff.shape[0] + 1, coeff.shape[1]))
        temp[:coeff.shape[0], :] = coeff
        temp[-1:,:] = coeff[0, :]
        coeff = temp

    if coeff.shape[1] % 2 != y_coeffs.highpasses[2][:, :, 0].shape[1] % 2:
        temp = np.zeros((coeff.shape[0], coeff.shape[1] + 1))
        temp[:, :coeff.shape[1]] = coeff
        temp[:,-1:] = coeff[:, 0].reshape((temp[:,-1:].shape[0], temp[:,-1:].shape[1]))
        coeff = temp


    for i in range(4):
        coeffs = np.zeros(lowpass_masks[i].shape)
        coeffs[
            height_up : height_up + coeff.shape[0],
            width_left : width_left + coeff.shape[1],
        ] = coeff

        v_coeffs.highpasses[2][:, :, i] += lowpass_str * (lowpass_masks[i] * coeffs)

    # lowpass_embed_subband = 1
    # coeff = wm_coeffs.lowpass
    # # print(coeff.shape) (68, 120)
    # h, w = coeff.shape
    # coeffs = np.zeros(lowpass_masks[lowpass_embed_subband].shape)
    # # coeffs[2 * lv1_h : 2 * lv1_h + h, 2 * lv1_w : 2 * lv1_w + w] = coeff
    # coeffs[height_up - 2 * lv1_h : height_up - 2 * lv1_h + h, width_left - 2 * lv1_w : width_left - 2 * lv1_w + w] = coeff
    # coeffs[height_up - 2 * lv1_h : height_up - 2 * lv1_h + h, width_right + 2 * lv1_w : width_right + 2 * lv1_w + w] = coeff
    # coeffs[height_down + 2 * lv1_h : height_down + 2 * lv1_h + h, width_left - 2 * lv1_w : width_left - 2 * lv1_w + w] = coeff
    # coeffs[height_down + 2 * lv1_h : height_down + 2 * lv1_h + h, width_right + 2 * lv1_w : width_right + 2 * lv1_w + w] = coeff
    # v_coeffs.highpasses[2][:, :, lowpass_embed_subband] += lowpass_str * (lowpass_masks[lowpass_embed_subband] * coeffs)

    img[:, :, 1] = img_transform.inverse(img_coeffs)
    img[:, :, 2] = img_transform.inverse(v_coeffs)
    ### embed watermark lowpass into V channel's highpass[2] end

    img = cv2.cvtColor(img, cv2.COLOR_YUV2BGR)
    img = np.clip(img, a_min=0, a_max=255)
    img = np.around(img).astype(np.uint8)

    return img

def decode_frame(wmed_img):
    wmed_img = cv2.cvtColor(wmed_img.astype(np.float32), cv2.COLOR_BGR2YUV)

    wmed_transform = dtcwt.Transform2d()
    wmed_coeffs = wmed_transform.forward(wmed_img[:, :, 1], nlevels=3)

    y_transform = dtcwt.Transform2d()
    y_coeffs = y_transform.forward(wmed_img[:, :, 0], nlevels=3)

    v_transform = dtcwt.Transform2d()
    v_coeffs = v_transform.forward(wmed_img[:, :, 2], nlevels=3)

    masks3 = [0 for i in range(6)]
    inv_masks3 = [0 for i in range(6)]
    shape3 = y_coeffs.highpasses[2][:, :, 0].shape
    for i in range(6):
        masks3[i] = cv2.filter2D(
            np.abs(y_coeffs.highpasses[1][:, :, i]),
            -1,
            np.array([[1 / 4, 1 / 4], [1 / 4, 1 / 4]]),
        )
        masks3[i] = np.ceil(rebin(masks3[i], shape3) * (1.0 / step))
        masks3[i][masks3[i] == 0] = 0.01
        masks3[i] *= 1.0 / max(12.0, np.amax(masks3[i]))
        inv_masks3[i] = 1.0 / masks3[i]

    shape = wmed_coeffs.highpasses[2][:, :, i].shape

    my_highpass = []
    h, w = (((shape[0] + 1) // 2 + 1) // 2 + 1) // 2, (
        ((shape[1] + 1) // 2 + 1) // 2 + 1
    ) // 2

    for i in range(wm_level):
        my_highpass.append(np.zeros((h, w, 6), dtype="complex_"))
        h = (h + 1) // 2
        w = (w + 1) // 2
        # print(my_highpass[i].shape)

    for i in range(6):
        coeffs = (wmed_coeffs.highpasses[2][:, :, i]) * inv_masks3[i] * 1 / highpass_str

        coeffs_height, coeffs_width = coeffs.shape
        height_moving_offset = 2 * my_highpass[-1].shape[0] + 1
        width_moving_offset = 2 * my_highpass[-1].shape[1] + 1
        

        if coeffs_height % 2 == 1:
            height_up = int(coeffs_height/2) - height_moving_offset
            height_down = int(coeffs_height/2) + 1 + height_moving_offset
        else:
            height_up = int(coeffs_height/2) - height_moving_offset
            height_down = int(coeffs_height/2) + height_moving_offset

        if coeffs_width % 2 == 1:
            width_left = int(coeffs_width/2) - width_moving_offset
            width_right = int(coeffs_width/2) + 1 + width_moving_offset
        else:
            width_left = int(coeffs_width/2) - width_moving_offset
            width_right = int(coeffs_width/2) + width_moving_offset

        for lv in range(wm_level):
            w = 0
            h = 0
            for m in range(lv):
                # print(my_highpass[m].shape)
                w += my_highpass[m].shape[1]
                h += my_highpass[m].shape[0]


            my_highpass[lv][:, :, i] = (
                coeffs[
                height_up - h - my_highpass[lv].shape[0] : height_up - h,
                width_left - w - my_highpass[lv].shape[1] : width_left - w,
                ]
                + coeffs[
                height_up - h - my_highpass[lv].shape[0] : height_up - h,
                width_right + w: width_right + w + my_highpass[lv].shape[1],
                ] 
                + coeffs[
                height_down + h: height_down + h + my_highpass[lv].shape[0] ,
                width_left - w -my_highpass[lv].shape[1] : width_left - w,
                ]
                + coeffs[
                height_down + h: height_down + h + my_highpass[lv].shape[0] ,
                width_right + w: width_right + w + my_highpass[lv].shape[1],
                ]
            )

    highpasses = tuple(my_highpass)

    ### extract watermark lowpass into V channel's highpass[2] (highpass as mask)
    lowpass_masks = [0 for i in range(6)]
    inv_lowpass_masks = [0 for i in range(6)]

    for i in range(4):
        lowpass_masks[i] = cv2.filter2D(
            np.abs(
                np.abs(y_coeffs.highpasses[1][:, :, i]),
            ),
            -1,
            np.array([[1 / 4, 1 / 4], [1 / 4, 1 / 4]]),
        )
        lowpass_masks[i] = np.ceil(
            rebin(lowpass_masks[i], v_coeffs.highpasses[2].shape) * (1 / step)
        )
        lowpass_masks[i] *= 1.0 / max(12.0, np.amax(lowpass_masks[i]))
        lowpass_masks[i][lowpass_masks[i] == 0] = 0.01
        inv_lowpass_masks[i] = 1.0 / lowpass_masks[i]

    lowpass = np.zeros(
        (my_highpass[-1].shape[0] * 2, my_highpass[-1].shape[1] * 2), dtype="complex_"
    )



    coeffs_height, coeffs_width = y_coeffs.highpasses[2][:, :, 0].shape

    if coeffs_height % 2 == 1:
        height_up = int(coeffs_height/2) - 1
        height_down = int(coeffs_height/2) 
    else:
        height_up = int(coeffs_height/2) - 1
        height_down = int(coeffs_height/2) - 1

    if coeffs_width % 2 == 1:
        width_left = int(coeffs_width/2) - 1
        width_right = int(coeffs_width/2)  
    else:
        width_left = int(coeffs_width/2) - 1
        width_right = int(coeffs_width/2) - 1
    
    for i in range(4):
        coeff = (v_coeffs.highpasses[2][:, :, i]) * inv_masks3[i] * 1 / lowpass_str
        lowpass[:, :] += coeff[
            height_up : height_up + lowpass.shape[0],
            width_left : width_left + lowpass.shape[1],
        ]


    # lowpass_embed_subband = 1
    # coeffs = (v_coeffs.highpasses[2][:, :, lowpass_embed_subband]) * inv_masks3[i] * 1 / lowpass_str
    # # print(coeff.shape) (68, 120)
    # lv1_h = my_highpass[0].shape[0]
    # lv1_w = my_highpass[0].shape[1]
    # h = 2 * my_highpass[-1].shape[0]
    # w = 2 * my_highpass[-1].shape[1]

    # lowpass[:, :] += (
    # coeffs[height_up - 2 * lv1_h : height_up - 2 * lv1_h + h, width_left - 2 * lv1_w : width_left - 2 * lv1_w + w] + 
    # coeffs[height_up - 2 * lv1_h : height_up - 2 * lv1_h + h, width_right + 2 * lv1_w : width_right + 2 * lv1_w + w] +
    # coeffs[height_down + 2 * lv1_h : height_down + 2 * lv1_h + h, width_left - 2 * lv1_w : width_left - 2 * lv1_w + w] +
    # coeffs[height_down + 2 * lv1_h : height_down + 2 * lv1_h + h, width_right + 2 * lv1_w : width_right + 2 * lv1_w + w]
    # )

    lowpass = lowpass.real.astype(np.float32)

    t = dtcwt.Transform2d()
    wm = t.inverse(dtcwt.Pyramid(lowpass, highpasses))

    avg_img = extract_and_average_small_arrays(wm, int(wm.shape[1]/2), int(wm.shape[0]/2))

    recovered_string = recover_string_from_image(bit_to_pixel, code_length, avg_img)
    return recovered_string, wm


# Encode a watermark.png into a cover image

In [39]:
coverimgpath = "frame_0000.png"
img = cv2.imread(coverimgpath)

wm = cv2.imread("image/wm.png")
wm = cv2.cvtColor(wm, cv2.COLOR_BGR2GRAY)
wm_transform = dtcwt.Transform2d()
wm_coeffs = wm_transform.forward(wm, nlevels=wm_level)


wm_img = embed_frame(img, wm_coeffs)
cv2.imwrite(f"image/wm_img.png", wm_img)


True

# Decode the watermark for wm_img.png

In [40]:
wm_image_path = "image/wm_img.png"

wmed_img = cv2.imread(wm_image_path)
recovered_string, recover_wm = decode_frame(wmed_img)

print(f'Embed Bitstream:    {random_binary_string}')
print(f'Extract Bitstream:  {recovered_string}')

cv2.imwrite(f'image/wm_extract.png', recover_wm)

Embed Bitstream:    111101010011000011011110010110011101100101000111110101010110
Extract Bitstream:  111101010011000011011110010110011001100101000111110101011110


True

# PSNR SSIM

In [41]:
# !ffmpeg -i frame_0000.png -i wm_img.png -filter_complex "psnr" -f null - 

In [42]:
# ! ffmpeg-quality-metrics video/life_300_wm.mp4 ../video/life_300.mp4 --metrics psnr ssim > result/life_300_wm_2d.json